[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiptos/generative/blob/main/notebooks/simulation_based_generator.ipynb)

# Simulation-based Generator

Simulation-based generators use logic to simulate processes over time, often with queues, events, or agents. The simpy library is great for modeling things like hospitals, factories, or service centers.

## Patients in a Clinic

This simulates patients arriving at a clinic, waiting for treatment, and leaving. The simulation generates timestamps and other synthetic data based on event timing:
* Patients arriving randomly
* Queuing for available doctors
* Treatment duration per patient
* Generated features: arrival time, wait time, treatment duration, etc.

__Use Cases__
* Queuing models in hospitals or banks
* Traffic simulations
* Customer service call centers
* Logistics & warehouse operations

In [ ]:
!pip install simpy

In [2]:
import simpy
import random
import pandas as pd

In [3]:
random.seed(42)

# Parameters
NUM_PATIENTS = 50
INTER_ARRIVAL_TIME = (3, 10)  # minutes between arrivals
TREATMENT_TIME = (5, 20)      # duration of treatment

# Store simulation data
records = []

def patient(env, name, doctor, records):
    arrival_time = env.now
    with doctor.request() as request:
        yield request
        wait_time = env.now - arrival_time
        treatment_duration = random.randint(*TREATMENT_TIME)
        yield env.timeout(treatment_duration)
        records.append({
            "Patient": name,
            "ArrivalTime": round(arrival_time, 2),
            "WaitTime": round(wait_time, 2),
            "TreatmentStart": round(env.now - treatment_duration, 2),
            "TreatmentEnd": round(env.now, 2),
            "TreatmentDuration": treatment_duration
        })

def patient_generator(env, doctor, records):
    for i in range(NUM_PATIENTS):
        yield env.timeout(random.randint(*INTER_ARRIVAL_TIME))
        env.process(patient(env, f"Patient_{i+1}", doctor, records))

# Setup environment
env = simpy.Environment()
doctor = simpy.Resource(env, capacity=2)  # two doctors available
env.process(patient_generator(env, doctor, records))
env.run()

# Create DataFrame
df = pd.DataFrame(records)
print(df.head())

# Optionally save
#df.to_csv("simulated_clinic_data.csv", index=False)

     Patient  ArrivalTime  WaitTime  TreatmentStart  TreatmentEnd  \
0  Patient_1            4         0               4            17   
1  Patient_2            7         0               7            19   
2  Patient_3           13         4              17            25   
3  Patient_5           22         3              25            32   
4  Patient_4           18         1              19            37   

   TreatmentDuration  
0                 13  
1                 12  
2                  8  
3                  7  
4                 18  
